In [ ]:
import pandas as pd
import jsonlines
import os
from glob import glob

In [ ]:
files = glob("../outputs/*/*/prometheus-13b-v1.0/alpaca-eval.json")
dfs = []

def filter_score(x):
    if x["score"] > 0:
        return x["score"]
    if "overall score is" in x["critic"]:
        score = int(x["critic"].split("overall score is")[1].split(".")[0].strip().replace(",","").split(" ", 1)[0])
        return score
    elif "a score of " in x["critic"]:
        score = x["critic"].split("a score of ")[1].split(" ")[0].split(".")[0].split(",")[0].strip()
        if score:
            return int(score)
        else:
            return -1
    else:
        return -1

for file in files:
    items = []
    for item in jsonlines.open(file):
        for turn in ["judge1", "judge2"]:
            scores = []
            for rubric, judgement in item[turn]:
                item[f"{turn}_{rubric}"] = filter_score(judgement)
        items.append(item)
        
    df = pd.DataFrame(items)
    dfs.append(df)

df = pd.concat(dfs)
df["output_len"] = df.output.apply(len)
df.shape